In [2]:
from keras.layers import *
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
import math
import numpy as np


In [21]:
(X_train,_),(_,_) = mnist.load_data()

In [22]:
X_train.shape,type(X_train)

((60000, 28, 28), numpy.ndarray)

In [23]:
X_train = X_train.reshape((*X_train.shape,1))
print(X_train.shape)

(60000, 28, 28, 1)


In [24]:
# Normalize between -1 and 1 
X_train = (X_train.astype('float32')-127.5)/127.5
print(np.max(X_train))
print(np.min(X_train))

1.0
-1.0


In [25]:
Noise_dim = 100
batch_size = 256
half_batches = 128
no_of_batches = int(X_train.shape[0]/batch_size)

In [27]:
# Define a generator
generator = Sequential()
generator.add(Dense(7*7*128,input_shape=(Noise_dim,)))
generator.add(Reshape((7,7,128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

# Double activation size 14x14x 64
generator.add(UpSampling2D())
generator.add(Conv2D(64,kernel_size=(5,5),padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

# Double activation size 28x28x 1
generator.add(UpSampling2D())
generator.add(Conv2D(1,kernel_size=(5,5),padding='same',activation='tanh'))

# Compile the model
generator.compile(loss='binary_crossentropy',optimizer='adam')
generator.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        204864    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 64)       

In [30]:
# Define the discriminator model
discriminator = Sequential()
discriminator.add(Conv2D(64,(5,5),strides=(2,2),padding='same',input_shape=(28,28,1)))
discriminator.add(LeakyReLU(0.2))

# Next Conv layer [ (14x14x64) --> (7x7x128) ]
discriminator.add(Conv2D(128,(5,5),strides=(2,2),padding='same'))
discriminator.add(LeakyReLU(0.2))

# Flatten
discriminator.add(Flatten())
discriminator.add(Dense(1,activation='sigmoid'))

# Compile the model
discriminator.compile(loss='binary_crossentropy',optimizer="adam")

discriminator.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6273      
Total params: 212,865
Trainable params: 212,865
Non-trainable params: 0
________________________________________________

In [31]:
!mkdir images_dc_gan
!mkdir models_dc_gan
!ls

DC-Gan Assignment.pdf    Online_MNIST_GAN_1.ipynb model
GAN_learning.ipynb       images                   models_dc_gan
MNIST_DC-GAN.ipynb       images_dc_gan


In [ ]:
# TR